In [ ]:
import tkinter as tk
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('employee.db')
c = conn.cursor()

# Create the employee table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS employees
             (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, department TEXT, position TEXT)''')
conn.commit()

# Function to add a new employee record to the database
def add_employee():
    name = name_entry.get()
    department = department_entry.get()
    position = position_entry.get()

    if name and department and position:
        c.execute("INSERT INTO employees (name, department, position) VALUES (?, ?, ?)",
                  (name, department, position))
        conn.commit()
        clear_entries()
        display_records()

# Function to display all employee records in the listbox
def display_records():
    records.delete(0, tk.END)
    c.execute("SELECT * FROM employees")
    for row in c.fetchall():
        records.insert(tk.END, f"ID: {row[0]}, Name: {row[1]}, Department: {row[2]}, Position: {row[3]}")

# Function to clear entry fields
def clear_entries():
    name_entry.delete(0, tk.END)
    department_entry.delete(0, tk.END)
    position_entry.delete(0, tk.END)

# Function to view details of the selected employee
def view_employee_details():
    selected_index = records.curselection()
    if selected_index:
        employee_id = selected_index[0] + 1  # Adjust for 1-based indexing
        c.execute("SELECT * FROM employees WHERE id=?", (employee_id,))
        employee = c.fetchone()
        if employee:
            details_label.config(text=f"ID: {employee[0]}, Name: {employee[1]}, Department: {employee[2]}, Position: {employee[3]}")
        else:
            details_label.config(text="Employee not found")

# Create the main window
root = tk.Tk()
root.title("Employee Management System")

# Create and place labels and entry widgets
name_label = tk.Label(root, text="Name:")
name_label.grid(row=0, column=0)
name_entry = tk.Entry(root)
name_entry.grid(row=0, column=1)

department_label = tk.Label(root, text="Department:")
department_label.grid(row=1, column=0)
department_entry = tk.Entry(root)
department_entry.grid(row=1, column=1)

position_label = tk.Label(root, text="Position:")
position_label.grid(row=2, column=0)
position_entry = tk.Entry(root)
position_entry.grid(row=2, column=1)

# Create and place buttons
add_button = tk.Button(root, text="Add Employee", command=add_employee)
add_button.grid(row=3, column=0, columnspan=2)

view_button = tk.Button(root, text="View Employee Details", command=view_employee_details)
view_button.grid(row=5, column=0, columnspan=2)

# Create and place a listbox for displaying records
records = tk.Listbox(root)
records.grid(row=4, column=0, columnspan=2)

# Create a label for displaying employee details
details_label = tk.Label(root, text="", wraplength=700)
details_label.grid(row=6, column=0, columnspan=2)

# Display initial records (if any)
display_records()

# Close the database connection when the application is closed
def on_closing():
    conn.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()
